In [1]:
import os

os.getcwd()

'/home/vmadmin/goqba/mk2/20230822'

In [7]:
# scikit-image 라이브러리 설치
!pip install scikit-image

  Using cached scikit_image-0.19.3-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (13.5 MB)
  Using cached imageio-2.31.1-py3-none-any.whl (313 kB)
  Using cached PyWavelets-1.3.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.4 MB)
  Using cached tifffile-2021.11.2-py3-none-any.whl (178 kB)
  Using cached networkx-2.6.3-py3-none-any.whl (1.9 MB)


### 데이터셋 분리
#### 기존 train에서 20퍼센트를 valid로

In [4]:
import os
import random
import shutil

def move_files(src_folder, dst_folder, file_list):
    for file_name in file_list:
        src_path = os.path.join(src_folder, file_name)
        dst_path = os.path.join(dst_folder, file_name)
        shutil.move(src_path, dst_path)

def main():
    RAW_folder = "/home/vmadmin/goqba/mk2/20230822/video/train"
    VAL_folder2 = "/home/vmadmin/goqba/mk2/20230822/video/valid"

    file_list = os.listdir(RAW_folder)
    random.shuffle(file_list)

    print(file_list)
    
    if not os.path.exists(VAL_folder2):
        os.makedirs(VAL_folder2)

    num_to_move_to_val = int(len(file_list) * 0.2)
    print(num_to_move_to_val)
    move_files(RAW_folder, VAL_folder2, file_list[:num_to_move_to_val])

if __name__ == "__main__":
    main()

['41591-1a.mp4', '71872-5a.mp4', '64500-2a.mp4', '52172-4a.mp4', '35143-3a.mp4', '68225-1a.mp4', '55311-5a.mp4', '42804-5a.mp4', '13260-2a.mp4', '64472-1a.mp4', '64500-5a.mp4', '73637-3a.mp4', '65405-5a.mp4', '49035-1a.mp4', '52797-2a.mp4', '49035-3a.mp4', '51529-3a.mp4', '59922-4a.mp4', '58121-4a.mp4', '73637-2a.mp4', '56912-2a.mp4', '41377-3a.mp4', '66476-3a.mp4', '25248-5a.mp4', '54649-3a.mp4', '61874-2a.mp4', '75558-5a.mp4', '60086-4a.mp4', '47378-5a.mp4', '44509-4a.mp4', '75217-5a.mp4', '68648-4a.mp4', '51773-2a.mp4', '72125-1a.mp4', '43046-1a.mp4', '50687-2a.mp4', '69614-2a.mp4', '57216-1a.mp4', '64768-3a.mp4', '73414-4a.mp4', '64472-2a.mp4', '71305-5a.mp4', '58377-4a.mp4', '41872-1a.mp4', '63868-1a.mp4', '64804-4a.mp4', '64501-5a.mp4', '52797-1a.mp4', '62065-1a.mp4', '47869-5a.mp4', '47378-2a.mp4', '50228-3a.mp4', '44509-2a.mp4', '57588-5a.mp4', '75342-1a.mp4', '50228-4a.mp4', '68648-3a.mp4', '51529-2a.mp4', '74249-2a.mp4', '25248-4a.mp4', '66357-4a.mp4', '71872-2a.mp4', '64804-

# 나뉘어진 train, test, valid 영상 폴더에 접근해 프레임으로 분할 후 각 지정 폴더에 저장
#### test의 경우 위에서 데이터셋 분리시 스킵했으므로 해당 코드 또한 valid 폴더로 넘어감

In [5]:
import os
import cv2
import natsort

def split_videos_in_folder(path):
    for i in range(3):
        # 월(month) 이후 접근할 폴더명들
        connect = ['train', 'test', 'valid']
            
        # 바운딩 영상을 가져올 폴더(input_folder 최종 예시: /home/vmadmin/goqba/mk2/20230822/video/train/)
        input_folder = path + '/video/' + connect[i] + '/'

        # 프레임을 저장할 폴더(output_folder 최종 예시: /home/vmadmin/goqba/mk2/20230822/frame/train/)
        output_folder = path + '/frame/' + connect[i] + '/'

        # 파일 정렬
        files = natsort.natsorted(os.listdir(input_folder))

        # 프레임을 저장할 폴더를 생성합니다.
        os.makedirs(output_folder, exist_ok=True)

        for file in files:
            frame_count = 1
            
            # 파일의 확장자를 가져옵니다.
            _, ext = os.path.splitext(file)

            # 파일이 영상 파일인지 확인합니다. (확장자가 .mp4, .avi 등)
            if ext.lower() in [".mp4", ".avi"]:
                # 영상 파일을 엽니다.
                video_path = os.path.join(input_folder, file)
                cap = cv2.VideoCapture(video_path)

                # 비디오 열기를 실패하면 다음 파일로 넘어갑니다.
                if not cap.isOpened():
                    print(f"{file} 비디오 열기 실패")
                    continue

                # 동영상 파일명에서 확장자 제거하여 저장할 이미지 파일명으로 사용합니다.
                video_name = os.path.splitext(file)[0]

                while True:
                    # 프레임을 읽습니다.
                    ret, frame = cap.read()
                    if not ret:
                        break

                    # 프레임을 이미지 파일로 저장합니다.
                    frame_path = os.path.join(output_folder, f"{video_name}_{frame_count}.jpg")
                    cv2.imwrite(frame_path, frame)

                    frame_count += 1

                cap.release()
                print(f"{file} 프레임 분할 완료")

# 기본 경로(video 폴더와 frame 폴더가 있는 경로)
path = "/home/vmadmin/goqba/mk2/20230822/"

def main():
    split_videos_in_folder(path)
    
if __name__ == "__main__":
    main()
    cv2.destroyAllWindows()

13260-1a.mp4 프레임 분할 완료
13260-3a.mp4 프레임 분할 완료
13260-4a.mp4 프레임 분할 완료
13260-5a.mp4 프레임 분할 완료
25248-2a.mp4 프레임 분할 완료
25248-3a.mp4 프레임 분할 완료
29110-2a.mp4 프레임 분할 완료
29110-4a.mp4 프레임 분할 완료
29110-5a.mp4 프레임 분할 완료
33078-1a.mp4 프레임 분할 완료
33078-2a.mp4 프레임 분할 완료
33078-4a.mp4 프레임 분할 완료
33078-5a.mp4 프레임 분할 완료
35143-1a.mp4 프레임 분할 완료
35143-2a.mp4 프레임 분할 완료
35143-4a.mp4 프레임 분할 완료
35143-5a.mp4 프레임 분할 완료
37468-1a.mp4 프레임 분할 완료
37468-2a.mp4 프레임 분할 완료
37468-4a.mp4 프레임 분할 완료
37468-5a.mp4 프레임 분할 완료
38264-1a.mp4 프레임 분할 완료
38264-2a.mp4 프레임 분할 완료
38264-3a.mp4 프레임 분할 완료
38264-4a.mp4 프레임 분할 완료
38264-5a.mp4 프레임 분할 완료
41377-1a.mp4 프레임 분할 완료
41377-2a.mp4 프레임 분할 완료
41377-4a.mp4 프레임 분할 완료
41377-5a.mp4 프레임 분할 완료
41591-2a.mp4 프레임 분할 완료
41591-3a.mp4 프레임 분할 완료
41591-4a.mp4 프레임 분할 완료
41591-5a.mp4 프레임 분할 완료
41872-2a.mp4 프레임 분할 완료
41872-3a.mp4 프레임 분할 완료
41872-4a.mp4 프레임 분할 완료
41872-5a.mp4 프레임 분할 완료
42804-1a.mp4 프레임 분할 완료
42804-2a.mp4 프레임 분할 완료
42804-3a.mp4 프레임 분할 완료
42804-4a.mp4 프레임 분할 완료
42933-1a.mp4 프레임 분할 완료
42933-2a.mp

75558-2a.mp4 프레임 분할 완료
75735-1a.mp4 프레임 분할 완료
75735-2a.mp4 프레임 분할 완료
75735-3a.mp4 프레임 분할 완료
75735-4a.mp4 프레임 분할 완료
75735-5a.mp4 프레임 분할 완료
13260-2a.mp4 프레임 분할 완료
25248-1a.mp4 프레임 분할 완료
25248-4a.mp4 프레임 분할 완료
25248-5a.mp4 프레임 분할 완료
29110-1a.mp4 프레임 분할 완료
29110-3a.mp4 프레임 분할 완료
35143-3a.mp4 프레임 분할 완료
37468-3a.mp4 프레임 분할 완료
41377-3a.mp4 프레임 분할 완료
41591-1a.mp4 프레임 분할 완료
41872-1a.mp4 프레임 분할 완료
42804-5a.mp4 프레임 분할 완료
43046-1a.mp4 프레임 분할 완료
43046-2a.mp4 프레임 분할 완료
44509-2a.mp4 프레임 분할 완료
44509-4a.mp4 프레임 분할 완료
46067-1a.mp4 프레임 분할 완료
47378-1a.mp4 프레임 분할 완료
47378-2a.mp4 프레임 분할 완료
47378-5a.mp4 프레임 분할 완료
47869-4a.mp4 프레임 분할 완료
47869-5a.mp4 프레임 분할 완료
49035-1a.mp4 프레임 분할 완료
49035-3a.mp4 프레임 분할 완료
49035-4a.mp4 프레임 분할 완료
50228-3a.mp4 프레임 분할 완료
50228-4a.mp4 프레임 분할 완료
50445-3a.mp4 프레임 분할 완료
50687-2a.mp4 프레임 분할 완료
51057-5a.mp4 프레임 분할 완료
51529-2a.mp4 프레임 분할 완료
51529-3a.mp4 프레임 분할 완료
51773-2a.mp4 프레임 분할 완료
52172-4a.mp4 프레임 분할 완료
52548-2a.mp4 프레임 분할 완료
52797-1a.mp4 프레임 분할 완료
52797-2a.mp4 프레임 분할 완료
54596-2a.mp

# 바운딩박스 이미지, 좌푯값 텍스트 파일 생성

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import pybboxes as pbx
import natsort
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.io import imread
from skimage.draw import rectangle_perimeter

def find_object(path):
    for i in range(3):
        # 월(month) 이후 접근할 폴더명들
        connect = ['train', 'test', 'valid']

        # 프레임을 가져올 폴더(input_folder 최종 예시: /home/vmadmin/goqba/mk2/20230822/frame/train/)
        input_folder = path + '/frame/' + connect[i] + '/'

        # 바운딩 박스 프레임을 저장할 폴더(output_folder 최종 예시: /home/vmadmin/goqba/mk2/20230822/bound/train/)
        output_folder = path + '/bound/' + connect[i] + '/'

        # 프레임 내 좌푯값을 저장할 텍스트의 폴더 경로(txt_folder 최종 예시: /home/vmadmin/goqba/mk2/20230822/loc/train/)
        txt_folder = path + '/loc/' + connect[i] + '/'
        
        # 입력 폴더의 모든 파일을 가져옵니다.
        files = natsort.natsorted(os.listdir(input_folder))

        # 프레임을 저장할 폴더를 생성합니다.
        os.makedirs(output_folder, exist_ok=True)

        # 텍스트를 저장할 폴더 생성
        os.makedirs(txt_folder, exist_ok=True)

        frame_count = 0
        
        # 블롭 검출에 사용될 매개변수 설정
        max_sigma = 30
        threshold = 0.1

        # 바운딩 박스들의 라벨 초기화
        box_labels = []

        for file in files:
            # 파일의 확장자를 가져옵니다.
            _, ext = os.path.splitext(file)

            # 파일이 이미지 파일인지 확인합니다. (확장자가 .jpg, .jpeg, png 등)
            if ext.lower() in [".jpg", ".jpeg", "png"]:

                # 이미지 파일 경로 결합
                image_path = os.path.join(input_folder, file)

                # 이미지 로드
                image = imread(image_path, as_gray=True)

                # 블롭 검출 (blob_dog 사용)
                blobs = blob_dog(image, max_sigma=max_sigma, threshold=threshold)
            
                # 텍스트 파일을 쓰기 모드로 열기
                with open(f"{txt_folder}{object_filename}.txt", "w") as file_txt:
                    
                    # 검출된 블롭마다 바운딩 박스 생성 및 좌표 저장
                    for idx, blob in enumerate(blobs):
                        y, x, sigma = blob
                        radius = sigma * np.sqrt(2) * 3  # 크기를 더 크게 설정 (5를 곱하여 조절)
        
                        # 바운딩 박스에 라벨값 0 적용
                        box_labels.append(0)
            
                        # 바운딩 박스의 좌표 계산
                        top_left = (int(x - radius), int(y - radius))
                        bottom_right = (int(x + radius), int(y + radius))
        
                        # 바운딩 박스의 너비와 높이 확인
                        box_width = bottom_right[0] - top_left[0]  # x 좌표에 대한 차이를 계산하여 너비 구함
                        box_height = bottom_right[1] - top_left[1]  # y 좌표에 대한 차이를 계산하여 높이 구함

                        if box_width < 20 or box_height < 20:  # 너비 또는 높이가 20보다 작으면 바운딩 박스 좌푯값을 배열 및 텍스트 파일에 저장함
                            # YOLO 좌표 변환 메서드 호출
                            voc_bbox = (top_left[0], top_left[1], bottom_right[0], bottom_right[1])
                            W, H = 640, 480 # WxH of the image
                            yolo_LOC = pbx.convert_bbox(voc_bbox, from_type="voc", to_type="yolo", image_size=(W,H))
            
                            # 바운딩 박스 정보를 텍스트 파일에 저장
                            file_txt.write(f"{box_labels[idx]} {yolo_LOC[0]} {yolo_LOC[1]} {yolo_LOC[2]} {yolo_LOC[3]}\n") 
            
                            # 실 이미지에 바운딩 박스를 칠할 좌푯값을 배열에 저장함
                            bounding_box_coords.append((top_left, bottom_right))
    
                    # 원본 이미지 로드
                    original_image = cv2.imread(image_path)
    
                    # 원본 이미지에 바운딩 박스 그리기
                    for box_coords in bounding_box_coords:
                        top_left, bottom_right = box_coords
                        cv2.rectangle(original_image, top_left, bottom_right, (0, 255, 0), 1)  # 초록색 바운딩 박스
    
                    # 결과 이미지 저장
                    cv2.imwrite(f"{output_folder}{object_filename}.jpg", original_image)

            else:
                print("이미지 파일이 아닙니다. 확장자를 확인해주세요")
            
    return box_labels

# 기본 경로(frame, bound, loc 폴더가 있는 최종 경로)
path = "/home/vmadmin/goqba/mk2/20230822/"

def main():    
    # 흰색 바운딩 박스 검출 및 표시
    find_object(path)
    
if __name__=="__main__":
    main()
    cv2.destroyAllWindows()